In [22]:
# Housekeeping, imports
import pandas as pd
import numpy as np
import os
from IPython.display import Markdown

In [23]:
# Create folders if needed
if not(os.path.exists('mapping')):
   os.mkdir('mapping')
for format in ['csv', 'md', 'json', 'xml', 'excel']:
    if not(os.path.exists(os.path.join('mapping', format))):
       os.mkdir(os.path.join('mapping', format))

In [24]:
# Housekeeping, imports
zibinfo = pd.read_excel('zib_versions.xlsx', sheet_name='table_info_zibs')
zibinfo = zibinfo.fillna('')
zibinfo.head()

,zib,wiki_URL_2020,wiki_URL_2017,OID_zib,sub_header_1,sub_header_2
0,Ademhaling,https://zibs.nl/wiki/Ademhaling-v3.2(2020NL),https://zibs.nl/wiki/Ademhaling-v3.1(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.12.5,,
1,Adresgegevens,https://zibs.nl/wiki/Adresgegevens-v1.1(2020NL),https://zibs.nl/wiki/Adresgegevens-v1.0(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.20.5,,
2,AlcoholGebruik,https://zibs.nl/wiki/AlcoholGebruik-v3.2(2020NL),https://zibs.nl/wiki/AlcoholGebruik-v3.1(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.7.3,,
3,Alert,https://zibs.nl/wiki/Alert-v4.1(2020NL),https://zibs.nl/wiki/Alert-v3.2(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.8.1,zie ZIB-1769 de leveranciers hebben gevraagd w...,zie ook https://www.rivm.nl/antibioticaresiste...
4,AlgemeneMeting,deprecated in 2020,https://zibs.nl/wiki/AlgemeneMeting-v3.0(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.13.3,De zib AlgemeneMeting komt te vervallen. Voor ...,


In [25]:
excels = os.listdir("mapping/excel/")
dfs = []
for excel in excels:
    dfs.append(pd.read_excel("mapping/excel/" + excel))
df = pd.concat(dfs)
df.head()

,ZibName,ConceptID_2017,ConceptName_2017,Codelists_2017,Change,ConceptID_2020,ConceptName_2020,Codelists_2020,Bits,Omschrijving,TypeChange,Impact_heen,TRANSLATIE_spec_heen,Impact_terug,TRANSLATIE_spec_terug
0,Ademhaling,NL-CM:12.5.1,Ademhaling,NaN,oranje: minor change,NL-CM:12.5.1,Ademhaling,NaN,ZIB-924,SNOMED CT DefintionCode concept aangepast,TERMINOLOGY MAPPING CONCEPT CHANGE,Medium,SCT DefintionCode [blank] -> [422834003 Evalua...,Medium,SCT DefintionCode [422834003 Evaluatie van ade...
1,Ademhaling,NL-CM:12.5.2,Ademfrequentie,NaN,groen: geen wijzigingen,NL-CM:12.5.2,Ademfrequentie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ademhaling,NL-CM:12.5.3,Toelichting,NaN,groen: geen wijzigingen,NL-CM:12.5.3,Toelichting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ademhaling,NL-CM:10.1.3,ProductType,NaN,groen: geen wijzigingen,NL-CM:10.1.3,ProductType,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ademhaling,NL-CM:12.5.4,AdemhalingDatumTijd,NaN,groen: geen wijzigingen,NL-CM:12.5.4,AdemhalingDatumTijd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Replace nulls with emtpy string
df = df.fillna('')
cols = df.columns
cols

Index(['ZibName', 'ConceptID_2017', 'ConceptName_2017', 'Codelists_2017',
       'Change', 'ConceptID_2020', 'ConceptName_2020', 'Codelists_2020',
       'Bits', 'Omschrijving', 'TypeChange', 'Impact_heen',
       'TRANSLATIE_spec_heen', 'Impact_terug', 'TRANSLATIE_spec_terug'],
      dtype='object')

In [27]:
# Make a list of zibs.
zibs = df['ZibName'].dropna().unique()
zibs

array(['Ademhaling', 'Adresgegevens', 'AlcoholGebruik', 'Alert',
       'AlgemeneMeting', 'AllergieIntolerantie', 'AnatomischeLocatie',
       'ApgarScore', 'BarthelIndex', 'BasisElementen',
       'BehandelAanwijzing', 'BehandelAanwijzing2', 'Behandeldoel',
       'Bereik', 'Betaler', 'Blaasfunctie', 'Bloeddruk', 'Brandwond',
       'BurgerlijkeStaat', 'ChecklistPijngedrag', 'ComfortScore',
       'Contact', 'Contactgegevens', 'Contactpersoon', 'Darmfunctie',
       'DAS', 'DecubitusWond', 'DOSScore', 'DrugsGebruik',
       'Familieanamnese', 'FarmaceutischProduct', 'FLACCpijnScore',
       'FunctieHoren', 'FunctieZien', 'FunctioneleOfMentaleStatus',
       'GebruiksInstructie', 'Gezinssituatie', 'GezinssituatieKind',
       'GlasgowComaScale', 'Hartfrequentie', 'Huidaandoening',
       'HulpVanAnderen', 'Infuus', 'JuridischeSituatie',
       'LaboratoriumUitslag', 'Levensovertuiging', 'Lichaamsgewicht',
       'Lichaamslengte', 'Lichaamstemperatuur', 'Medicatieafspraak',
       'Medi

In [28]:
# Markdown. Make a header.
smallcols = ['ConceptID_2017', 'ConceptName_2017', 'Change', 'TypeChange', 'Impact_heen', 'TRANSLATIE_spec_heen', 'Impact_terug', 'TRANSLATIE_spec_terug',  'Omschrijving']
header = "# {zibname}\n## File formats\n\nThe translation specs are available as: \n[CSV](../csv/{zibname}.csv) [JSON](../json/{zibname}.json) [XML](../xml/{zibname}.xml)\n\n"
subheader = ""
shcols = ['ConceptID_2017', 'ConceptName_2017', 'Change', 'TypeChange', 'Omschrijving']
for zib in zibs:
    if len(zib) == 0:
        continue
    tdf = df[df['ZibName'] == zib]
    tdf = tdf.fillna('')
    url = zibinfo[zibinfo['zib'] == zib]['wiki_URL_2017'].item()
    subheader = "[" + url.split('/')[-1] + "](" + url +")\n\n"
    url = zibinfo[zibinfo['zib'] == zib]['wiki_URL_2020'].item()
    subheader = subheader + "[" + url.split('/')[-1] + "](" + url +")\n\n"
    subheader = subheader + "\n\n" + zibinfo[zibinfo['zib'] == zib]['sub_header_1'].item()
    subheader = subheader + "\n\n" + zibinfo[zibinfo['zib'] == zib]['sub_header_2'].item()
    htable = tdf[(tdf['ConceptID_2017'].str.contains("DCM::"))][shcols]
    if not(htable.empty):
        mdhtable = '## Zib-level changes\n\n' + htable.to_markdown(index=False)
    else:
        mdhtable = ''
    mapping = tdf[~tdf['ConceptID_2017'].str.contains("DCM::")][cols].to_markdown(index=False)
    changes = tdf[~tdf['ConceptID_2017'].str.contains("DCM::") & ~tdf['Change'].str.contains("groen")][smallcols].to_markdown(index=False)
    with open(os.path.join('mapping', 'md', zib + '.md'), 'w', encoding='utf8') as file:
        content = header.format(zibname = zib) + '\n\n' + subheader + '\n\n' + mdhtable + '\n\n## Changes\n\n' + changes  + '\n\n## Mapping\n\n' + mapping + '\n\n'
        file.write(content)
Markdown(content)

Ademhaling
Adresgegevens
AlcoholGebruik
Alert
AlgemeneMeting
AllergieIntolerantie
AnatomischeLocatie
ApgarScore
BarthelIndex
BasisElementen
BehandelAanwijzing
BehandelAanwijzing2
Behandeldoel
Bereik
Betaler
Blaasfunctie
Bloeddruk
Brandwond
BurgerlijkeStaat
ChecklistPijngedrag
ComfortScore
Contact
Contactgegevens
Contactpersoon
Darmfunctie
DAS
DecubitusWond
DOSScore
DrugsGebruik
Familieanamnese
FarmaceutischProduct
FLACCpijnScore
FunctieHoren
FunctieZien
FunctioneleOfMentaleStatus
GebruiksInstructie
Gezinssituatie
GezinssituatieKind
GlasgowComaScale
Hartfrequentie
Huidaandoening
HulpVanAnderen
Infuus
JuridischeSituatie
LaboratoriumUitslag
Levensovertuiging
Lichaamsgewicht
Lichaamslengte
Lichaamstemperatuur
Medicatieafspraak
MedicatieContraIndicatie
MedicatieToediening2
Medicatieverstrekking
MedischHulpmiddel
Mobiliteit
MUSTScore
Naamgegevens
Nationaliteit
O2Saturatie
OntwikkelingKind
Opleiding
OverdrachtGeplandeZorgActiviteit
ParticipatieInMaatschappij
Patient
Patientbespreking
Pijnkenm

# Zwangerschap
## File formats

The translation specs are available as: 
[CSV](../csv/Zwangerschap.csv) [JSON](../json/Zwangerschap.json) [XML](../xml/Zwangerschap.xml)



[Zwangerschap-v3.1(2017NL)](https://zibs.nl/wiki/Zwangerschap-v3.1(2017NL))

[Zwangerschap-v4.0(2020NL)](https://zibs.nl/wiki/Zwangerschap-v4.0(2020NL))









## Changes

| ConceptID_2017   | ConceptName_2017               | Change                | TypeChange                         | Impact_heen   | TRANSLATIE_spec_heen                                                                                                                | Impact_terug   | TRANSLATIE_spec_terug                                                                                                                  | Omschrijving                                                                  |
|:-----------------|:-------------------------------|:----------------------|:-----------------------------------|:--------------|:------------------------------------------------------------------------------------------------------------------------------------|:---------------|:---------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------|
| NL-CM:7.14.1     | Zwangerschap                   | oranje: minor change  | TERMINOLOGY MAPPING CONCEPT CHANGE | Medium        | SCT DefinitionCode [364320009 Pregnancy observable] -> [118185001 bevinding betreffende zwangerschap]                               | Medium         | SCT DefinitionCode [118185001 bevinding betreffende zwangerschap] -> [364320009 Pregnancy observable]                                  | SNOMED CT DefintionCode concept aangepast                                     |
| NL-CM:7.14.2     | Zwanger                        | rood: major change    | CONCEPT REMOVED                    | High          | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2020 ontvanger]                   | Low            |                                                                                                                                        | Boolean concept zwanger is verwijderd.                                        |
| NL-CM:7.14.3     | ATermeDatum                    | geel: patch wijziging | SIMPLE PATCH CHANGE (EN)           | Low           |                                                                                                                                     | Low            |                                                                                                                                        | Kleine tekstwijzing in engelse vertaling "term date" naar ''due date''.       |
| NL-CM:7.14.5     | Graviditeit                    | oranje: minor change  | TERMINOLOGY MAPPING CONCEPT CHANGE | Medium        | SCT DefintionCode [blank] -> [161732006 Gravida]; LOINC DefintionCode[11996-6 Pregnancies] ->[blank]                                | Medium         | SCT DefintionCode [161732006 Gravida] ->[blank] ; LOINC DefintionCode [blank] -> [11996-6 Pregnancies]                                 | SNOMED CT en LOINC DefintionCodes concept aangepast                           |
| NL-CM:7.14.6     | Pariteit                       | oranje: minor change  | TERMINOLOGY MAPPING CONCEPT CHANGE | Medium        | SCT DefintionCode [blank] -> [364325004 Pariteit]; LOINC DefintionCode[11977-6 Parity] ->[blank]                                    | Medium         | SCT DefintionCode [364325004 Pariteit] -> [blank] ; LOINC DefintionCode [blank] -> [11977-6 Parity]                                    | SNOMED CT en LOINC DefintionCodes concept aangepast                           |
| NL-CM:7.14.7     | Toelichting                    | oranje: minor change  | TERMINOLOGY MAPPING CONCEPT CHANGE | Medium        | SCT DefintionCode [blank] -> [161714006 Estimated date of delivery]; LOINC DefintionCode[11778-8 Delivery date Estimated] ->[blank] | Medium         | SCT DefintionCode [161714006 Estimated date of delivery] -> [blank] ; LOINC DefintionCode [blank] -> [11778-8 Delivery date Estimated] | SNOMED CT en LOINC DefintionCodes concept aangepast                           |
| NL-CM:7.14.8     | DatumLaatsteMenstruatie        | rood: major change    | CONCEPT ADDITION                   | Low           |                                                                                                                                     | High           | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2017 ontvanger]                      | AtermeDatum container met onderliggende elementen zijn toegevoegd aan de zib. |
| NL-CM:7.14.9     | **concept toegevoegd in 2020** | rood: major change    | CONCEPT ADDITION                   | Low           |                                                                                                                                     | High           | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2017 ontvanger]                      | AtermeDatum container met onderliggende elementen zijn toegevoegd aan de zib. |
| NL-CM:7.14.10    | **concept toegevoegd in 2020** | rood: major change    | CONCEPT ADDITION                   | Low           |                                                                                                                                     | High           | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2017 ontvanger]                      | AtermeDatum container met onderliggende elementen zijn toegevoegd aan de zib. |
| NL-CM:7.14.11    | **concept toegevoegd in 2020** | rood: major change    | CONCEPT ADDITION                   | Low           |                                                                                                                                     | High           | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2017 ontvanger]                      | AtermeDatum container met onderliggende elementen zijn toegevoegd aan de zib. |

## Mapping

| ZibName      | ConceptID_2017   | ConceptName_2017               | Codelists_2017   | Change                  | ConceptID_2020   | ConceptName_2020               | Codelists_2020            | Bits     | Omschrijving                                                                  | TypeChange                         | Impact_heen   | TRANSLATIE_spec_heen                                                                                                                | Impact_terug   | TRANSLATIE_spec_terug                                                                                                                  |
|:-------------|:-----------------|:-------------------------------|:-----------------|:------------------------|:-----------------|:-------------------------------|:--------------------------|:---------|:------------------------------------------------------------------------------|:-----------------------------------|:--------------|:------------------------------------------------------------------------------------------------------------------------------------|:---------------|:---------------------------------------------------------------------------------------------------------------------------------------|
| Zwangerschap | NL-CM:7.14.1     | Zwangerschap                   |                  | oranje: minor change    | NL-CM:7.14.1     | Zwangerschap                   |                           | ZIB-1201 | SNOMED CT DefintionCode concept aangepast                                     | TERMINOLOGY MAPPING CONCEPT CHANGE | Medium        | SCT DefinitionCode [364320009 Pregnancy observable] -> [118185001 bevinding betreffende zwangerschap]                               | Medium         | SCT DefinitionCode [118185001 bevinding betreffende zwangerschap] -> [364320009 Pregnancy observable]                                  |
| Zwangerschap | NL-CM:7.14.2     | Zwanger                        |                  | rood: major change      | NL-CM:7.14.2     | **concept verwijderd in 2020** |                           | ZIB-890  | Boolean concept zwanger is verwijderd.                                        | CONCEPT REMOVED                    | High          | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2020 ontvanger]                   | Low            |                                                                                                                                        |
| Zwangerschap | NL-CM:7.14.3     | ATermeDatum                    |                  | geel: patch wijziging   | NL-CM:7.14.3     | ATermeDatum                    |                           | ZIB-907  | Kleine tekstwijzing in engelse vertaling "term date" naar ''due date''.       | SIMPLE PATCH CHANGE (EN)           | Low           |                                                                                                                                     | Low            |                                                                                                                                        |
| Zwangerschap | NL-CM:7.14.4     | Zwangerschapsduur              |                  | groen: geen wijzigingen | NL-CM:7.14.4     | Zwangerschapsduur              |                           |          |                                                                               |                                    |               |                                                                                                                                     |                |                                                                                                                                        |
| Zwangerschap | NL-CM:7.14.5     | Graviditeit                    |                  | oranje: minor change    | NL-CM:7.14.5     | Graviditeit                    |                           | ZIB-890  | SNOMED CT en LOINC DefintionCodes concept aangepast                           | TERMINOLOGY MAPPING CONCEPT CHANGE | Medium        | SCT DefintionCode [blank] -> [161732006 Gravida]; LOINC DefintionCode[11996-6 Pregnancies] ->[blank]                                | Medium         | SCT DefintionCode [161732006 Gravida] ->[blank] ; LOINC DefintionCode [blank] -> [11996-6 Pregnancies]                                 |
| Zwangerschap | NL-CM:7.14.6     | Pariteit                       |                  | oranje: minor change    | NL-CM:7.14.6     | Pariteit                       |                           | ZIB-890  | SNOMED CT en LOINC DefintionCodes concept aangepast                           | TERMINOLOGY MAPPING CONCEPT CHANGE | Medium        | SCT DefintionCode [blank] -> [364325004 Pariteit]; LOINC DefintionCode[11977-6 Parity] ->[blank]                                    | Medium         | SCT DefintionCode [364325004 Pariteit] -> [blank] ; LOINC DefintionCode [blank] -> [11977-6 Parity]                                    |
| Zwangerschap | NL-CM:7.14.7     | Toelichting                    |                  | oranje: minor change    | NL-CM:7.14.7     | Toelichting                    |                           | ZIB-890  | SNOMED CT en LOINC DefintionCodes concept aangepast                           | TERMINOLOGY MAPPING CONCEPT CHANGE | Medium        | SCT DefintionCode [blank] -> [161714006 Estimated date of delivery]; LOINC DefintionCode[11778-8 Delivery date Estimated] ->[blank] | Medium         | SCT DefintionCode [161714006 Estimated date of delivery] -> [blank] ; LOINC DefintionCode [blank] -> [11778-8 Delivery date Estimated] |
| Zwangerschap | NL-CM:7.14.8     | DatumLaatsteMenstruatie        |                  | rood: major change      | NL-CM:7.14.8     | DatumLaatsteMenstruatie        |                           | ZIB-890  | AtermeDatum container met onderliggende elementen zijn toegevoegd aan de zib. | CONCEPT ADDITION                   | Low           |                                                                                                                                     | High           | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2017 ontvanger]                      |
| Zwangerschap | NL-CM:7.14.9     | **concept toegevoegd in 2020** |                  | rood: major change      | NL-CM:7.14.9     | ATermeDatumItems               |                           | ZIB-890  | AtermeDatum container met onderliggende elementen zijn toegevoegd aan de zib. | CONCEPT ADDITION                   | Low           |                                                                                                                                     | High           | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2017 ontvanger]                      |
| Zwangerschap | NL-CM:7.14.10    | **concept toegevoegd in 2020** |                  | rood: major change      | NL-CM:7.14.10    | BepalingsMethode               | BepalingsMethodeCodelijst | ZIB-890  | AtermeDatum container met onderliggende elementen zijn toegevoegd aan de zib. | CONCEPT ADDITION                   | Low           |                                                                                                                                     | High           | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2017 ontvanger]                      |
| Zwangerschap | NL-CM:7.14.11    | **concept toegevoegd in 2020** |                  | rood: major change      | NL-CM:7.14.11    | DatumBepaling                  |                           | ZIB-890  | AtermeDatum container met onderliggende elementen zijn toegevoegd aan de zib. | CONCEPT ADDITION                   | Low           |                                                                                                                                     | High           | IF [blank]source->target ELSE [toon en stuur de inhoud van dit data item als vrije tekst naar een 2017 ontvanger]                      |



In [29]:
for zib in zibs:
    if len(zib) == 0:
        continue
    tdf = df[df['ZibName'] == zib]
    tdf.to_csv(os.path.join('mapping', 'csv', zib + '.csv'), index=False)
    tdf.to_xml(os.path.join('mapping', 'xml', zib + '.xml'), index=False)
    tdf.to_json(os.path.join('mapping', 'json', zib + '.json'), orient='records')

In [30]:
mdindex = "# Index of available zib translations\n\n"
for zib in sorted(zibs):
    mdindex = mdindex + "* [ " + zib + "](mapping/md/" + zib + ".md)\n"
with open('index.md', 'w', encoding='utf8') as file:
    file.write(mdindex)